In [3]:
import pandas as pd
from cyvcf2 import VCF

In [4]:
vcf = list(VCF('/Users/quinlan/Documents/Git/SEMIColon/data/PD34199_chr2_chunk2_snps_chunk.vcf'))


In [ ]:
variant = vcf[0]
# *** variant.POS gives you vcf coordinate; variant.start and variant.end give you bed coordinates)
print(variant.CHROM, variant.POS, variant.start, variant.end, variant.REF, variant.ALT)
# "Number of samples with data"
print(variant.INFO.get('NS'))
# Total read depth at the locus"
print(variant.INFO.get('DP'))
# depth by base
print(variant.INFO.get('DPB'))

print(variant.format("GT"))

# "Total number of alleles in called genotypes"
print(variant.INFO.get('AN'))



In [22]:
phases = variant.gt_phases
quals = variant.gt_quals,  # numpy array

In [26]:


# Print the Genotype Quality (GQ)
print(f"Genotype Quality (GQ), Phred-scaled marginal probability of the called genotype: {variant.format('GQ')}")

# Print the Genotype Likelihood (GL)
print(f"Genotype Likelihood (GL), log10-scaled likelihoods of the data given the called genotype: {variant.format('GL')}")

# Print the Read Depth (DP)
print(f"Read Depth (DP), the total number of reads covering this site: {variant.format('DP')}")

# Print the Number of Observations for each Allele (AD)
print(f"Allele Depth (AD), number of observations for each allele (reference and alternate): {variant.format('AD')}")

# Print the Reference Allele Observation Count (RO)
print(f"Reference Allele Observation Count (RO), the number of reads supporting the reference allele: {variant.format('RO')}")

# Print the Sum of Quality of Reference Observations (QR)
print(f"Sum of Quality of Reference Observations (QR), sum of quality scores for reference allele reads: {variant.format('QR')}")

# Print the Alternate Allele Observation Count (AO)
print(f"Alternate Allele Observation Count (AO), the number of reads supporting the alternate allele: {variant.format('AO')}")

# Print the Sum of Quality of Alternate Observations (QA)
print(f"Sum of Quality of Alternate Observations (QA), sum of quality scores for alternate allele reads: {variant.format('QA')}")



Genotype (GT): ['\x02\x02' '\x02\x02' '\x02\x02' '\x02\x02' '\x02\x02' '\x02\x04'
 '\x02\x02' '\x02\x02' '\x02\x02' '\x02\x02']
Genotype Quality (GQ), Phred-scaled marginal probability of the called genotype: None
Genotype Likelihood (GL), log10-scaled likelihoods of the data given the called genotype: [[  0.       -5.71957 -60.8677   -5.71957 -60.8677  -60.8677 ]
 [  0.       -5.11751 -56.8304   -5.11751 -56.8304  -56.8304 ]
 [  0.       -3.91339 -47.5957   -3.91339 -47.5957  -47.5957 ]
 [  0.       -4.81648 -52.1476   -2.21863 -49.6177  -49.0157 ]
 [  0.       -5.41854 -53.0129   -5.41854 -53.0129  -53.0129 ]
 [ -2.24021   0.      -37.6155   -5.85257 -38.2176  -43.7241 ]
 [  0.       -7.22472 -82.7304   -7.22472 -82.7304  -82.7304 ]
 [  0.       -5.11751 -58.0003   -5.11751 -58.0003  -58.0003 ]
 [  0.       -7.82678 -87.2063   -7.82678 -87.2063  -87.2063 ]
 [  0.       -8.12781 -91.1764   -8.12781 -91.1764  -91.1764 ]]
Read Depth (DP), the total number of reads covering this site: [[

In [21]:
# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
genotypes = variant.gt_types
alt_depths = variant.gt_alt_depths
ref_depths = variant.gt_ref_depths
bases = variant.gt_bases

In [ ]:
# "Number of samples with data"
print("Number of samples with data:", variant.INFO.get('NS'))

# "Total read depth at the locus"
print("Total read depth at the locus:", variant.INFO.get('DP'))

# "Total read depth per base"
print("Total read depth per base:", variant.INFO.get('DPB'))

# "Total number of alternate alleles in called genotypes"
print("Total number of alternate alleles:", variant.INFO.get('AC'))

# "Total number of alleles in called genotypes"
print("Total number of alleles:", variant.INFO.get('AN'))

# "Estimated allele frequency"
print("Estimated allele frequency:", variant.INFO.get('AF'))

# "Reference allele observation count (full)"
print("Reference allele observation count (full):", variant.INFO.get('RO'))

# "Alternate allele observation count (full)"
print("Alternate allele observation count (full):", variant.INFO.get('AO'))

# "Reference allele observation count (partial)"
print("Reference allele observation count (partial):", variant.INFO.get('PRO'))

# "Alternate allele observation count (partial)"
print("Alternate allele observation count (partial):", variant.INFO.get('PAO'))

# "Reference allele quality sum"
print("Reference allele quality sum (phred):", variant.INFO.get('QR'))

# "Alternate allele quality sum"
print("Alternate allele quality sum (phred):", variant.INFO.get('QA'))

# "Reference allele quality sum (partial)"
print("Reference allele quality sum (partial, phred):", variant.INFO.get('PQR'))

# "Alternate allele quality sum (partial)"
print("Alternate allele quality sum (partial, phred):", variant.INFO.get('PQA'))

# "Number of reference observations on the forward strand"
print("Reference observations (forward strand):", variant.INFO.get('SRF'))

# "Number of reference observations on the reverse strand"
print("Reference observations (reverse strand):", variant.INFO.get('SRR'))

# "Number of alternate observations on the forward strand"
print("Alternate observations (forward strand):", variant.INFO.get('SAF'))

# "Number of alternate observations on the reverse strand"
print("Alternate observations (reverse strand):", variant.INFO.get('SAR'))


In [ ]:
# "Strand balance probability for the reference allele"
print("Strand balance probability (reference allele):", variant.INFO.get('SRP'))

# "Strand balance probability for the alternate allele"
print("Strand balance probability (alternate allele):", variant.INFO.get('SAP'))

# "Allele balance at heterozygous sites"
print("Allele balance (heterozygous sites):", variant.INFO.get('AB'))

# "Allele balance probability at heterozygous sites"
print("Allele balance probability (heterozygous sites):", variant.INFO.get('ABP'))

# "Run length: number of consecutive repeats of the alternate allele in the reference genome"
print("Run length (alternate allele):", variant.INFO.get('RUN'))

# "Read Placement Probability for alternate alleles"
print("Read Placement Probability (alternate allele):", variant.INFO.get('RPP'))

# "Read Placement Probability for reference observations"
print("Read Placement Probability (reference observations):", variant.INFO.get('RPPR'))

# "Reads Placed Left: supporting alternate allele"
print("Reads Placed Left (supporting alternate):", variant.INFO.get('RPL'))

# "Reads Placed Right: supporting alternate allele"
print("Reads Placed Right (supporting alternate):", variant.INFO.get('RPR'))

# "End Placement Probability for alternate alleles"
print("End Placement Probability (alternate allele):", variant.INFO.get('EPP'))

# "End Placement Probability for reference observations"
print("End Placement Probability (reference observations):", variant.INFO.get('EPPR'))

# "Alternate allele depth ratio"
print("Alternate allele depth ratio:", variant.INFO.get('DPRA'))

# "Log odds ratio of best genotype combination"
print("Log odds ratio of best genotype combination:", variant.INFO.get('ODDS'))

# "Number of genotyping iterations to convergence or bailout"
print("Genotyping iterations to convergence or bailout:", variant.INFO.get('GTI'))

# "Type of allele (snp, mnp, ins, del, complex)"
print("Type of allele:", variant.INFO.get('TYPE'))

# "Extended CIGAR representation of alternate allele"
print("CIGAR representation of alternate allele:", variant.INFO.get('CIGAR'))

# "Number of unique non-reference alleles"
print("Number of unique non-reference alleles:", variant.INFO.get('NUMALT'))

# "Mean number of non-reference allele observations per sample"
print("Mean number of non-reference allele observations per sample:", variant.INFO.get('MEANALT'))

# "Allele length"
print("Allele length:", variant.INFO.get('LEN'))

# "Mean mapping quality of observed alternate alleles"
print("Mean mapping quality (alternate alleles):", variant.INFO.get('MQM'))

# "Mean mapping quality of observed reference alleles"
print("Mean mapping quality (reference alleles):", variant.INFO.get('MQMR'))

# "Proportion of observed alternate alleles supported by properly paired reads"
print("Proportion of alternate alleles supported by paired reads:", variant.INFO.get('PAIRED'))

# "Proportion of observed reference alleles supported by properly paired reads"
print("Proportion of reference alleles supported by paired reads:", variant.INFO.get('PAIREDR'))

# "Minimum depth in gVCF output block"
print("Minimum depth in gVCF output block:", variant.INFO.get('MIN_DP'))

# "Last position in gVCF output record"
print("Last position in gVCF output record:", variant.INFO.get('END'))

# "Fraction of alternate allele observations in reads from Illumina"
print("Fraction of alternate observations from Illumina reads:", variant.INFO.get('technology.Illumina'))


In [ ]:
from collections import defaultdict

# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
HOM_REF=0
HET=1
UNKNOWN=2
HOM_ALT=3

# Initialize the dictionary to hold the count of each configuration
config_counts = defaultdict(int)

# Loop through each variant in the VCF
for v in vcf:
    # Get the genotypes for the samples
    genotypes = v.gt_types

    # Assign sample context


    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") >= 12:
        if blood == youngsperm == oldsperm:
            continue
        elif blood == UNKNOWN:
            continue
        elif youngsperm == oldsperm == UNKNOWN:
            continue
        elif blood == youngsperm and oldsperm == UNKNOWN:
            continue
        elif blood == oldsperm and youngsperm == UNKNOWN:
            continue
        else:
    # Create a tuple of the genotype configuration
            config = (youngsperm, blood, oldsperm)

    # Increment the count for this configuration
            config_counts[config] += 1


# Print the configuration counts
for config, count in config_counts.items():
    print(f"{config}: {count} occurrences")

In [5]:
def get_info_v(v, blood_col, sperm1_col, sperm2_col):
    genotypes = v.gt_types
    alt_depths = v.gt_alt_depths
    ref_depths = v.gt_ref_depths
    bases = v.gt_bases

    #assign sample context: GT
    youngsperm = genotypes[sperm1_col]
    blood = genotypes[blood_col]
    oldsperm = genotypes[sperm2_col]

    #assign sample context: AD
    ys_altdepth = alt_depths[sperm1_col]
    os_altdepth = alt_depths[sperm2_col]
    blood_altdepth = alt_depths[blood_col]

    #assign sample context: ref_depths
    ys_refdepth = ref_depths[sperm1_col]
    os_refdepth = ref_depths[sperm2_col]
    blood_refdepth = ref_depths[blood_col]

    #assign sample context: ref_depths
    ys_bases = bases[sperm1_col]
    os_bases = bases[sperm2_col]
    blood_bases = bases[blood_col]

    return youngsperm, blood, oldsperm, ys_altdepth, os_altdepth, blood_altdepth, ys_refdepth, os_refdepth, blood_refdepth, ys_bases, os_bases, blood_bases

def get_interesting(v, youngsperm, blood, oldsperm):
    if blood == youngsperm == oldsperm:
        return False
    elif blood == UNKNOWN:
        return False
    elif youngsperm == oldsperm == UNKNOWN:
        return False
    elif blood == youngsperm and oldsperm == UNKNOWN:
        return False
    elif blood == oldsperm and youngsperm == UNKNOWN:
        return False
    else:
        return True

In [ ]:
# gt_types is array of 0,1,2,3==HOM_REF, HET, UNKNOWN, HOM_ALT
HOM_REF=0
HET=1
UNKNOWN=2
HOM_ALT=3

blood_col = 1
sperm1_col = 0
sperm2_col = 2

# variant.gt_types, variant.gt_ref_depths, variant.gt_alt_depths # numpy arrays
# variant.gt_phases, variant.gt_quals, variant.gt_bases # numpy array

for v in vcf:
    youngsperm, blood, oldsperm, ys_altdepth, os_altdepth, blood_altdepth, ys_refdepth, os_refdepth, blood_refdepth, ys_bases, os_bases, blood_bases = get_info_v(v,
                                                                                                                        blood_col, sperm1_col, sperm2_col)
    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") > 10 and get_interesting(v, youngsperm, blood, oldsperm):
        print("blood", blood_bases, "alt depth", blood_altdepth, "ref depth", blood_refdepth, "young sperm", ys_bases, "alt depth", ys_altdepth,
                  "ref depth", ys_refdepth, "old sperm",os_bases, "alt depth", os_altdepth, "ref depth", os_refdepth)
        #print("blood", "sperm1", "sperm2")
        #print(v.INFO.get("AC"))
            #print(v.gt_types, ys_altdepth, os_altdepth, v.QUAL)
            #print(os_refdepth, ys_refdepth, blood_refdepth)

In [ ]:
for v in vcf:
    # Get the necessary info
    genotypes = v.gt_types

    # Assign sample context: GT
    youngsperm = genotypes[0]
    blood = genotypes[1]
    oldsperm = genotypes[2]

    # Access FORMAT fields
    ro_blood = v.format('RO')[1]
    ao_blood = v.format('AO')[1]

    ro_youngsperm = v.format('RO')[0]
    ao_youngsperm = v.format('AO')[0]

    ro_oldsperm = v.format('RO')[2]
    ao_oldsperm = v.format('AO')[2]

    if v.QUAL>20 and v.INFO.get("NS") > 1 and v.INFO.get("DP") > 10 and get_interesting(v, youngsperm, blood, oldsperm):

    # Print the FORMAT fields
        print(f"Blood - RO: {ro_blood}, AO: {ao_blood}")
        print(f"Young Sperm - RO: {ro_youngsperm}, AO: {ao_youngsperm}")
        print(f"Old Sperm - RO: {ro_oldsperm}, AO: {ao_oldsperm}")

